# High-level API overview

https://www.ovito.org/docs/current/python/introduction/overview.html

In [1]:
import ovito
print(ovito.__file__)

/jet/home/nhew/.conda/envs/ovito-env/lib/python3.10/site-packages/ovito/__init__.py


## Importing data from disk

In [2]:
from ovito.io import import_file
pipeline = import_file("dump_nemd_analyze")

In [3]:
print(pipeline.source)

## Applying modifiers

In [4]:
from ovito.modifiers import *
#pipeline.modifiers.append(ColorCodingModifier(property = 'Potential Energy'))
#pipeline.modifiers.append(SliceModifier(normal = (0,0,1)))

In [5]:
#pipeline.modifiers.append(CommonNeighborAnalysisModifier(cutoff=3.2))

In [6]:
#modifier = CommonNeighborAnalysisModifier()
#modifier.cutoff = 3.2
#modifier.only_selected = True
#pipeline.modifiers.append(modifier)

## Exporting data to disk

In [7]:
from ovito.io import export_file
export_file(pipeline, "outputdata.dump", "lammps/dump",
    columns = ["Position.X", "Position.Y", "Position.Z"])

## Accessing computation results

In [8]:
data = pipeline.compute()
print(data.particles.keys())

KeysView(Particles())


In [9]:
data.objects

[AttributeDataObject(), AttributeDataObject(), AttributeDataObject(), Particles(), SimulationCell()]

In [10]:
print(data.cell[...])

[[ 5.96496302e+01 -1.55431223e-15  0.00000000e+00 -1.47341512e+00]
 [ 0.00000000e+00  5.96496302e+01  0.00000000e+00 -1.47341512e+00]
 [ 0.00000000e+00  0.00000000e+00  5.96496302e+01 -1.47341512e+00]]


In [11]:
print(data.particles.positions[...])

[[ 9.5892053  -1.33128556  1.14226888]
 [ 2.37924713 12.46044025  3.07371795]
 [ 1.3195953   0.09585928 -1.46997793]
 ...
 [57.92562703 50.71959379 56.86195482]
 [50.82654558 57.57763108 55.39499146]
 [50.01548956 54.5019172  57.71786736]]


## Accessing a pipeline's input data

In [12]:
input_data = pipeline.source.compute()
output_data = pipeline.compute()

## Rendering images and movies

In [13]:
#pipeline = import_file("dump_nemd_analyze")
pipeline.add_to_scene()

In [14]:
import math 
from ovito.vis import Viewport

vp = Viewport()
vp.type = Viewport.Type.Perspective
vp.camera_pos = (-100, -150, 150)
vp.camera_dir = (2, 3, -3)
vp.fov = math.radians(60.0)

In [15]:
# Causes Jupyter Notebook to crash
#vp.render_image(filename="myimage.png", size=(800,600))

# File I/O

https://www.ovito.org/docs/current/python/introduction/file_io.html

## Data import

In [18]:
from ovito.io import import_file
pipeline = import_file("dump_nemd_analyze")

In [19]:
print(pipeline)

In [21]:
# Re-use an existing pipeline, but load a different input file
pipeline.source.load("dump_nemd_analyze") 

## Loading simulation trajectories

# Radial distribution functions

In [26]:
from ovito.io import import_file
from ovito.modifiers import CoordinationAnalysisModifier

The following Python program demonstrates how to load a particle configuration, compute the RDF using the modifier and export the data to a text file:

In [136]:
# Load a particle dataset, apply the modifier, and evaluate pipeline.
# Only applies to the first frame
pipeline = import_file("0.25")
modifier = CoordinationAnalysisModifier(cutoff = 3.2, number_of_bins = 200, partial=True)
pipeline.modifiers.append(modifier)

frame = 0
data = pipeline.compute(0)

In [137]:
# Print the computed g(r) function values.
print(data.tables['coordination-rdf'].xy())

[[0.008      0.         0.         0.        ]
 [0.024      0.         0.         0.        ]
 [0.04       0.         0.         0.        ]
 [0.056      0.         0.         0.        ]
 [0.072      0.         0.         0.        ]
 [0.088      0.         0.         0.        ]
 [0.104      0.         0.         0.        ]
 [0.12       0.         0.         0.        ]
 [0.136      0.         0.         0.        ]
 [0.152      0.         0.         0.        ]
 [0.168      0.         0.         0.        ]
 [0.184      0.         0.         0.        ]
 [0.2        0.         0.         0.        ]
 [0.216      0.         0.         0.        ]
 [0.232      0.         0.         0.        ]
 [0.248      0.         0.         0.        ]
 [0.264      0.         0.         0.        ]
 [0.28       0.         0.         0.        ]
 [0.296      0.         0.         0.        ]
 [0.312      0.         0.         0.        ]
 [0.328      0.         0.         0.        ]
 [0.344      

In [139]:
from dfttk.plotly_format import plot_format
import plotly.graph_objects as go

# Access the RDF data
rdf_data = data.tables['coordination-rdf'].xy()
r_values = rdf_data[:, 0]  # Radial distance values

fig = go.Figure()

# Loop over all g(r) columns (skip the first column, which is r)
for i in range(1, rdf_data.shape[1]):
    fig.add_trace(go.Scatter(x=r_values, y=rdf_data[:, i], mode='lines', name=f'g(r) {i}'))

plot_format(fig, "Pair separation distance (Å)", "g(r)")
fig.show()

In [142]:
import numpy as np

particle_id = data.particles['Particle Identifier'].array
particle_type = data.particles['Particle Type'].array
coord_numbers = data.particles['Coordination'].array

# Map particle type numbers to element names
type_names = np.where(particle_type == 1, 'Al', np.where(particle_type == 2, 'Cu', particle_type))

# Combine type name, particle id, and coordination number into a table
coord_table = np.column_stack((type_names, particle_id, coord_numbers))
coord_table = coord_table[coord_table[:, 1].astype(float).argsort()] # Sort by particle ID (now column 1)
print(coord_table)

[['Cu' '1' '8']
 ['Al' '2' '7']
 ['Al' '3' '7']
 ...
 ['Al' '10974' '9']
 ['Cu' '10975' '8']
 ['Al' '10976' '6']]


The following code demonstrates how to use the `TimeAveragingModifier` to compute the RDF for every frame of an MD simulation and
generate a time-averaged RDF histogram. Finally, the RDF histogram is written to an output file.

In [143]:
from ovito.io import import_file, export_file
from ovito.modifiers import CoordinationAnalysisModifier, TimeAveragingModifier
import numpy

In [153]:
# Load a simulation trajectory consisting of several frames:
pipeline = import_file("0.25")
print("Number of MD frames:", pipeline.num_frames)

Number of MD frames: 1001


In [154]:
# Insert the RDF calculation modifier into the pipeline:
pipeline.modifiers.append(CoordinationAnalysisModifier(cutoff = 3.2, number_of_bins = 200, partial=True))

In [155]:
# Insert the time-averaging modifier into the pipeline, which accumulates
# the instantaneous DataTable produced by the previous modifier and computes a mean histogram.
pipeline.modifiers.append(TimeAveragingModifier(operate_on='table:coordination-rdf'))

In [156]:
total_rdf = pipeline.compute().tables['coordination-rdf[average]'].xy()
total_rdf

array([[8.00000000e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.40000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.00000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.60000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.20000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.80000000e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.04000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.20000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.36000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.52000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.68000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.84000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.00000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.16000000e-01, 0.00000000e+00, 0.00000000e

In [157]:
# Plot the computed g(r) function values using plotly
from dfttk.plotly_format import plot_format
import plotly.graph_objects as go

# Access the RDF data
rdf_data = total_rdf
r_values = rdf_data[:, 0]  # Radial distance values

fig = go.Figure()

# Loop over all g(r) columns (skip the first column, which is r)
for i in range(1, rdf_data.shape[1]):
    fig.add_trace(go.Scatter(x=r_values, y=rdf_data[:, i], mode='lines', name=f'g(r) {i}'))

plot_format(fig, "Pair separation distance (Å)", "g(r)")
fig.show()

In [158]:
# Compute the average coordination numbers for all particles over the entire trajectory

num_frames = pipeline.num_frames
all_particle_types = []
all_particle_ids = []
all_coord_numbers = [] 
modifier = CoordinationAnalysisModifier(cutoff = 3.2, number_of_bins = 200, partial=True)
pipeline.modifiers.append(modifier)

for frame in range(num_frames):
    data = pipeline.compute(frame)
    particle_id = data.particles['Particle Identifier'].array
    particle_type = data.particles['Particle Type'].array
    coord_numbers = data.particles['Coordination'].array
    
    coord_table = np.column_stack((particle_type, particle_id, coord_numbers))
    coord_table = coord_table[coord_table[:, 1].argsort()] # Sort in ascending order by particle ID
    
    all_particle_types.append(coord_table[:, 0])
    all_particle_ids.append(coord_table[:, 1])
    all_coord_numbers.append(coord_table[:, 2])

In [159]:
# Check that all particle types and IDs are the same across frames
# Convert list of arrays to a 2D NumPy array
all_particle_types_array = np.array(all_particle_types)
all_particle_ids_array = np.array(all_particle_ids)

# Check if all rows (frames) are identical to the first row
all_same = np.all(np.all(all_particle_types_array == all_particle_types_array[0], axis=1))
all_same = np.all(np.all(all_particle_ids_array == all_particle_ids_array[0], axis=1))

print("All particle type arrays are identical across frames:", all_same)
print("All particle ID arrays are identical across frames:", all_same)

All particle type arrays are identical across frames: True
All particle ID arrays are identical across frames: True


In [167]:
# Get the average coordination numbers across all frames for each particle 
all_coord_numbers_array = np.array(all_coord_numbers)
average_coord_numbers = np.mean(all_coord_numbers_array, axis=0)
average_coord_numbers

# Map particle type numbers to element names for the first frame
type_names = np.where(all_particle_types_array[0] == 1, 'Al', np.where(all_particle_types_array[0] == 2, 'Cu', all_particle_types_array[0]))

average_coord_table = np.column_stack((type_names, all_particle_ids_array[0].astype(int), average_coord_numbers))
average_coord_table

array([['Cu', '1', '7.968031968031968'],
       ['Al', '2', '7.894105894105894'],
       ['Al', '3', '7.78021978021978'],
       ...,
       ['Al', '10974', '7.841158841158841'],
       ['Cu', '10975', '7.929070929070929'],
       ['Al', '10976', '7.937062937062937']], dtype='<U32')

In [41]:
# Data export method 1: Convert to NumPy array and write data to a text file:
total_rdf = pipeline.compute().tables['coordination-rdf[average]'].xy()
numpy.savetxt("rdf.txt", total_rdf)

In [42]:
# Data export method 2: Use OVITO's own export function for DataTable objects:
export_file(pipeline, "rdf.txt", "txt/table", key="coordination-rdf[average]")

In [49]:
#This code example demonstrates how to access the partial RDFs computed by the modifier
from ovito.io import import_file
from ovito.modifiers import CoordinationAnalysisModifier

# Load input data
pipeline = import_file("dump_nemd_analyze")

# Print the list of input particle types
# They are represented by ParticleType objects attached to the \'Particle Type\' particle property.
for t in pipeline.compute().particles.particle_types.types:
    print(f"Type {t.id}: {t.name}")

# Calculate partial RDFs:
pipeline.modifiers.append(CoordinationAnalysisModifier(cutoff=3.2, number_of_bins=200, partial=True))

# Access the output DataTable:
rdf_table = pipeline.compute().tables['coordination-rdf']

# The y-property of the data points of the DataTable is now a vectorial property.
# Each vector component represents one partial RDF.
rdf_names = rdf_table.y.component_names     

# Print a list of partial g(r) functions.
for component, name in enumerate(rdf_names):
    print("g(r) for pair-wise type combination %s:" % name)
    print(rdf_table.y[:,component])

# The DataTable.xy() method yields everything as one combined NumPy table.
# This includes the 'r' values in the first array column, followed by the
# tabulated g(r) partial functions in the remaining columns.
print(rdf_table.xy())


Type 1: 
g(r) for pair-wise type combination 1-1:
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.   